<a href="https://colab.research.google.com/github/manuel1729/Simulaci-n-1/blob/main/Inventario_Simulaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import numpy as np
import random as rd
from scipy.stats import t
import matplotlib.pyplot as plt

In [23]:
# PARÁMETROS Y DISTRIBUCIONES DEL PROBLEMA
COSTO_ORDENAR = 100
COSTO_INVENTARIO_ANUAL = 20  # $/unidad/año
COSTO_INVENTARIO_MES = COSTO_INVENTARIO_ANUAL / 12  # $1.67/unidad/mes
COSTO_FALTANTE = 50  # $/unidad
INVENTARIO_INICIAL = 150  # unidades
MESES = 12

In [14]:
# Factores estacinales durante el año
factores_estacionales = [
    1.20, 1.00, 0.90, 0.80, 0.80, 0.70,
    0.80, 0.90, 1.00, 1.20, 1.30, 1.40]

# Demanda y tiempo entrega
def demanda_trans_inver():
  r = rd.random()
  if r < 0.01: return 35
  elif 0.010 <= r and r < 0.025: return 36
  elif 0.025 <= r and r < 0.045: return 37
  elif 0.045 <= r and r < 0.065: return 38
  elif 0.065 <= r and r < 0.087: return 39
  elif 0.087 <= r and r < 0.110: return 40
  elif 0.110 <= r and r < 0.135: return 41
  elif 0.135 <= r and r < 0.162: return 42
  elif 0.162 <= r and r < 0.190: return 43
  elif 0.190 <= r and r < 0.219: return 44
  elif 0.219 <= r and r < 0.254: return 45
  elif 0.254 <= r and r < 0.299: return 46
  elif 0.299 <= r and r < 0.359: return 47
  elif 0.359 <= r and r < 0.424: return 48
  elif 0.424 <= r and r < 0.494: return 49
  elif 0.494 <= r and r < 0.574: return 50
  elif 0.574 <= r and r < 0.649: return 51
  elif 0.649 <= r and r < 0.719: return 52
  elif 0.719 <= r and r < 0.784: return 53
  elif 0.784 <= r and r < 0.844: return 54
  elif 0.844 <= r and r < 0.894: return 55
  elif 0.894 <= r and r < 0.934: return 56
  elif 0.934 <= r and r < 0.964: return 57
  elif 0.964 <= r and r < 0.980: return 58
  elif 0.980 <= r and r < 0.995: return 59
  else: return 60

def generar_tiempo_entrega():
    r = rd.random()
    if r < 0.3:
        return 1
    elif r < 0.7:
        return 2
    else:
        return 3

In [24]:
#Simulamos un año el inventario
def simular_12(q, R, seed=None):
    if seed is not None:
        np.random.seed(seed)

    inventario = INVENTARIO_INICIAL
    orden_pendiente = False
    mes_llegada_orden = None
    registros = []

    # Costos acumulados inicimos en 0
    costo_ordenar_anual = 0
    costo_inventario_anual = 0
    costo_faltante_anual = 0

    for mes in range(MESES):
        #órdenes pendientes
        if orden_pendiente and mes == mes_llegada_orden:
            inventario += q
            orden_pendiente = False

        #Calculamos demanda ajustada
        demanda_base = demanda_trans_inver()
        demanda_ajustada = round(demanda_base * factores_estacionales[mes])
        inventario_inicial = inventario

        #Satisfacemos demanda y calculamos el faltante
        inventario -= demanda_ajustada
        faltante = max(0, -inventario)

        # Ajustamos inventario si hay faltante
        if inventario < 0:
            inventario = 0

        #Calculamos inventario promedio del mes
        inventario_prom = (inventario_inicial + inventario) / 2

        #Calculamos los costos del mes
        costo_inv_mes = inventario_prom * COSTO_INVENTARIO_MES
        costo_faltante_mes = faltante * COSTO_FALTANTE
        costo_orden_mes = 0

        #Ordenamos si es necesario
        if inventario <= R and not orden_pendiente:
            tiempo_entrega = generar_tiempo_entrega()
            mes_llegada_orden = mes + tiempo_entrega
            orden_pendiente = True
            costo_orden_mes = COSTO_ORDENAR

        # Registramos todo lo snterior por mes en un diccionario
        registros.append({
            'Mes': mes + 1,
            'Inventario Inicial': inventario_inicial,
            'Demanda Base': demanda_base,
            'Factor Estacional': factores_estacionales[mes],
            'Demanda Ajustada': demanda_ajustada,
            'Inventario Final': inventario,
            'Faltante': faltante,
            'Orden Colocada': 1 if costo_orden_mes > 0 else 0,
            'Costo Orden': costo_orden_mes,
            'Costo Inventario': costo_inv_mes,
            'Costo Faltante': costo_faltante_mes
        })

        # Acumulamos los costos anuales
        costo_ordenar_anual += costo_orden_mes
        costo_inventario_anual += costo_inv_mes
        costo_faltante_anual += costo_faltante_mes

    # Calculamos el costo total anual
    costo_total_anual = costo_ordenar_anual + costo_inventario_anual + costo_faltante_anual

    return {
        'Registros': pd.DataFrame(registros),
        'Costo Ordenar Anual': costo_ordenar_anual,
        'Costo Inventario Anual': costo_inventario_anual,
        'Costo Faltante Anual': costo_faltante_anual,
        'Costo Total Anual': costo_total_anual
    }

In [25]:
def simular_multiple(q, R, n, seed=None):
    #Simula múltiples años
    costos_totales = []

    for i in range(n):
        current_seed = seed + i if seed is not None else None
        resultados = simular_12(q, R, current_seed)
        costos_totales.append(resultados['Costo Total Anual'])

    return np.mean(costos_totales)

In [79]:
#Ejecutamos con datos del ejercicio.
if __name__ == "__main__":
    #Simulamos con los valores iniciales
    q_ini, R_ini = 200, 100
    resultados_ini = simular_12(q_ini, R_ini, seed=42)

    print("RESULTADOS SIMULACIÓN VALORES INICIALES PARA UN AÑO (q=200, R=100)")
    print("="*60)
    print(f"Costo total anual: ${resultados_ini['Costo Total Anual']:,.2f}")
    print(f" - Costo de ordenar: ${resultados_ini['Costo Ordenar Anual']:,.2f}")
    print(f" - Costo de inventario: ${resultados_ini['Costo Inventario Anual']:,.2f}")
    print(f" - Costo de faltante: ${resultados_ini['Costo Faltante Anual']:,.2f}")

    print("\nDetalle mensual:")
    print(resultados_ini['Registros'].to_string(index=False))

    #Simulamos 10 múltiples años para estabilizar
    n = 5
    costo_promedio = simular_multiple(q_ini, R_ini, n, seed=42)
    print(f"\nCosto promedio anual ({n} años): ${costo_promedio:,.2f}")

RESULTADOS SIMULACIÓN VALORES INICIALES PARA UN AÑO (q=200, R=100)
Costo total anual: $4,785.00
 - Costo de ordenar: $300.00
 - Costo de inventario: $2,535.00
 - Costo de faltante: $1,950.00

Detalle mensual:
 Mes  Inventario Inicial  Demanda Base  Factor Estacional  Demanda Ajustada  Inventario Final  Faltante  Orden Colocada  Costo Orden  Costo Inventario  Costo Faltante
   1                 150            39                1.2                47               103         0               0            0        210.833333               0
   2                 103            60                1.0                60                43         0               1          100        121.666667               0
   3                 243            52                0.9                47               196         0               0            0        365.833333               0
   4                 196            48                0.8                38               158         0               0    

In [80]:
def simular_costo_promedio(q, R, n, seed=None, alpha=0.05):
    costos = []

    for i in range(n):
        semilla = seed + i if seed is not None else None
        resultados = simular_12(q, R, semilla)
        costos.append(resultados["Costo Total Anual"])

    media = np.mean(costos)
    std = np.std(costos, ddof=1)
    error_est = std / np.sqrt(n)

    t_crit = t.ppf(1 - alpha/2, df=n-1)
    margen_error = t_crit * error_est

    ic_inf = media - margen_error
    ic_sup = media + margen_error

    return {
        "Media": media,
        "Desviación estándar": std,
        "Error estándar": error_est,
        "Intervalo de confianza": (ic_inf, ic_sup),
        "Costos simulados": costos
    }

In [83]:
resultado_mc = simular_costo_promedio(q_ini, R_ini, n=100, seed=42)
print(f"Costo promedio estimado: ${resultado_mc['Media']:,.2f}")
print(f"Desviación estándar: ${resultado_mc['Desviación estándar']:,.2f}")
print(f"IC 95%: (${resultado_mc['Intervalo de confianza'][0]:,.2f}, ${resultado_mc['Intervalo de confianza'][1]:,.2f})")
resultado_mc = simular_costo_promedio(q_ini, R_ini, n=100, seed=42)
print(f"Costo promedio estimado: ${resultado_mc['Media']:,.2f}")

Costo promedio estimado: $3,797.62
Desviación estándar: $847.61
IC 95%: ($3,629.44, $3,965.81)
Costo promedio estimado: $3,969.93
